In [1]:
# Importing necessary libraries
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm import tqdm
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os

/Users/georgestavrakis/opt/anaconda3/envs/LLMZoomcamp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load environment variables from .envrc for the Chat GPT
load_dotenv("../.envrc")

True

In [6]:
# Load the QA dataset to index
data = pd.read_csv('../data/investment_data.csv') # Could be the sample 
records = data.to_dict(orient='records')
# Load the ground truth dataset
ground_truth_df = pd.read_csv('../retrieval_evaluation/ground_truth.csv')
ground_truth = ground_truth_df.to_dict(orient='records')

## Setting up the Optimal Retieval Part

In [8]:
# Initialize the selected model to create the embeddings
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# Create the embeddings for each record in our QA dataset
for record in tqdm(records):
    # Extract the text fields you want to embed along with threir combinations
    question_answer_context = record['question'] + ' ' + record['answer'] + ' ' + record['context']
    # Create the embedding for each text field
    record['question_answer_context_vector'] = model.encode(question_answer_context)

/Users/georgestavrakis/opt/anaconda3/envs/LLMZoomcamp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 6990/6990 [04:40<00:00, 24.90it/s]


In [11]:
# Initialize the client 
es_client = Elasticsearch('http://localhost:9200')

# Create the Schema of the Elastic Search Index for vector search
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "context": {"type": "text"},
            "ticker": {"type": "keyword"}, 
            "company": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_answer_context_vector": {
                "type": "dense_vector",
                "dims": 384,             
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

# Provide the name of the index
index_name = "investment-info"
# Check if the index exists
if es_client.indices.exists(index=index_name):
    # Delete the existing index
    es_client.indices.delete(index=index_name)
# Create the elastic search index
response = es_client.indices.create(index=index_name, body=index_settings)
# Verify that elastic search is created
print(response)

# Fetch all the documents into the elastic search index
for record in tqdm(records):
    es_client.index(index = index_name, document=record)

# Adjusting the ES query for hybrid search with the best of two options
def document_search(query, vector, company, a = 0.75):
    # This is the query for the vector search with the best field 
    vector_query = {
        "field": 'question_answer_context_vector',
        "query_vector": vector, # This will recieve a vector of the user query
        "k": 5,
        "num_candidates": 10000,
        "boost": a, # Here you can set up the weight the vector search will have in the results
        "filter": {
            "term": {
                "company": company
            }
        }
    }
    # This is the query for the keyword search with best boosting
    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query, # This will recieve the user query itself
                    "fields": ["question", "answer", "context"],
                    "type": "best_fields",
                    "boost": 1-a, # Here you can set up the weight the keyword search will have in the results
                }
            },
            "filter": {
                "term": {
                    "company": company
                }
            }
        }
    }
    # Here is the combination of the two search methods
    search_query = {
        "knn": vector_query,
        "query": keyword_query,
        "size": 5,   # This is the number of the returned documents
        "_source": ['question', 'answer', 'context', 'ticker' ,'company' ,'id'] # The fields that will be returned for each retrieved document 
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'investment-info'}


100%|██████████| 6990/6990 [00:24<00:00, 289.58it/s]


In [16]:
# Create the function will generate the context
def context_generation(retrieved_docs):
    # Initialize context
    context = ''
    # Create context for each document
    for record in retrieved_docs:
        context += 'question: {question} answer: {answer} \n\n'.format(**record)
    # Return context
    return context

# Create the prompt template
prompt_template = '''
You are an Investment Assistant with deep expertise in financial markets.
Provide a fact-based answer to the user's QUESTION strictly using the information provided in the CONTEXT. 
Do not include opinions or external knowledge.

QUESTION: {question}

CONTEXT: 
{context}
'''.strip()

# Create a function to create the prompt
def prompt_generation(query, retrieved_docs, prompt_template = prompt_template):
    # Create the context
    prompt_context = {}
    prompt_context['question'] = query
    prompt_context['context'] = context_generation(retrieved_docs)
    # Generate the prompt
    prompt = prompt_template.format(**prompt_context)
    return prompt

In [15]:
# Initialize the openai instance
client = OpenAI()

# Create the function to request an answer from an LLM
def ask_llm(prompt, model='gpt-4o-mini'):
    # Create the request body and make the request
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
# Create the RAG function combining all the above information
def investment_assistant(model, query, prompt_template = prompt_template):
    # Retrieve the relevant documents
    relevant_docs = document_search(query)
    # Create the prompt for the model
    prompt = prompt_generation(query, relevant_docs, prompt_template = prompt_template)
    # Ask the LLM model
    response = ask_llm(prompt)
    return response

### Examine different GPT models to Enchance RAG Results

In [12]:
# Selecting 2 affortable models to examine
models = ['gpt-3.5-turbo','gpt-4o-mini']

## Perform Prompt engineering to Enchance RAG Results